In [289]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [235]:
# Load Data
base = 'store-sales-time-series-forecasting/'
df_holidays = pd.read_csv(base +"holidays_events.csv")
df_oil = pd.read_csv(base+'oil.csv')
df_stores = pd.read_csv(base+'stores.csv')
df_test = pd.read_csv(base + 'test.csv')
df_train = pd.read_csv(base + 'train.csv')
df_trans = pd.read_csv(base + 'transactions.csv')

# Data Preparation

## Formatting and Merging

In [236]:
# Change to datetime
df_holidays.date = pd.to_datetime(df_holidays.date)
df_oil.date = pd.to_datetime(df_oil.date)
df_train.date = pd.to_datetime(df_train.date)
df_test.date = pd.to_datetime(df_test.date)
df_trans.date = pd.to_datetime(df_train.date)

In [273]:
df_train1 = df_train.copy()
df_train1 = df_train1.merge(df_stores, on = 'store_nbr', how = 'left')
df_train1 = df_train1.merge(df_oil, on = 'date', how = 'left')
# Ignore trans dataset - makes no sense.
#df_train1 = df_train1.merge(df_trans, on = ['date','store_nbr'], how = 'left')

# Handling Holidays
'''
Need to drop instances where holidays have been transferred and handle corner cases where the are 
multiple holidays on the same day. National = relevant to all, regional only relevant to state,
local only relevant to city.
'''

df_effective_holidays = df_holidays[df_holidays['transferred'] == False]

drop_cols = ['type','locale','transferred']

df_local = df_effective_holidays[df_effective_holidays['locale'] == 'Local']\
            .drop_duplicates(subset=['date','locale_name'])\
            .drop(columns = drop_cols)\
            .rename(columns = {'description':'local_holiday'})

df_regional = df_effective_holidays[df_effective_holidays['locale'] == 'Regional']\
            .drop_duplicates(subset=['date','locale_name'])\
            .drop(columns = drop_cols)\
            .rename(columns = {'description':'regional_holiday'})
    
df_national = df_effective_holidays[df_effective_holidays['locale'] == 'National']\
            .drop_duplicates(subset='date')\
            .drop(columns = drop_cols)\
            .rename(columns = {'description':'national_holiday'})

df_train1 = pd.merge(df_train1, df_local, how = 'left', left_on = ['date','city'], right_on = ['date', 'locale_name'])
df_train1 = pd.merge(df_train1, df_regional, how = 'left', left_on = ['date','state'], right_on = ['date', 'locale_name'])
df_train1 = pd.merge(df_train1, df_national, how = 'left', left_on = 'date', right_on = 'date')
df_train1 = df_train1.drop(columns = ['locale_name_x','locale_name_y','locale_name'])

In [274]:
# Note we will start with a simplified version that only considers whether holidays are happening or not - not which specific holiday is happening.
# this is done to control dimensionality.
df_train1['national_holiday'] = df_train1['national_holiday'].apply(lambda x: 0 if type(x) == float else 1)
df_train1['regional_holiday'] = df_train1['regional_holiday'].apply(lambda x: 0 if type(x) == float else 1)
df_train1['local_holiday'] = df_train1['local_holiday'].apply(lambda x: 0 if type(x) == float else 1)

In [275]:
df_train1

,id,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,dcoilwtico,local_holiday,regional_holiday,national_holiday
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0,Quito,Pichincha,D,13,NaN,0,0,1
1,1,2013-01-01,1,BABY CARE,0.000,0,Quito,Pichincha,D,13,NaN,0,0,1
2,2,2013-01-01,1,BEAUTY,0.000,0,Quito,Pichincha,D,13,NaN,0,0,1
3,3,2013-01-01,1,BEVERAGES,0.000,0,Quito,Pichincha,D,13,NaN,0,0,1
4,4,2013-01-01,1,BOOKS,0.000,0,Quito,Pichincha,D,13,NaN,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3000883,3000883,2017-08-15,9,POULTRY,438.133,0,Quito,Pichincha,B,6,47.57,0,0,0
3000884,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,Quito,Pichincha,B,6,47.57,0,0,0
3000885,3000885,2017-08-15,9,PRODUCE,2419.729,148,Quito,Pichincha,B,6,47.57,0,0,0
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,Quito,Pichincha,B,6,47.57,0,0,0


# Additional Cleaning

## Categorical to Numerical Conversion

In [278]:
df_train1 = pd.get_dummies(df_train1, columns = ['family', 'city', 'state', 'type', 'cluster'])

## Generate Lags
Let's try lags for sales, onpromotion, dcoilwtico at 1, 3, 7, 14, and 30 days.

In [287]:
lags = [1,3,7,14,30]
cols = ['sales','onpromotion','dcoilwtico']

df_train1 = df_train1.assign(**{
        f'{col} (t-{lag})': df_train1[col].shift(lag)
        for lag in lags
        for col in cols
})

In [ ]:
df_train1

In [290]:
df_train1.plot('dcoilwtico')

KeyboardInterrupt: 

ValueError: Date ordinal 3150931.35 converts to 10596-12-15T08:24:00.000000 (using epoch 1970-01-01T00:00:00), but Matplotlib dates must be between year 0001 and 9999.